In [7]:
import pandas as pd
import os
import glob
from Bio import SeqIO
import csv

To run this notebook, the working directory must contain three subdirectories:

1. interpro_outputs – InterProScan outputs for all processed (merged_nr) sequences.
2. interpro_refs_outputs – InterProScan outputs for all reference sequences.
3. merged_nr_fastas – FASTA files of all processed sequences in the database.

## Filter 1 - Selecting sequences that have the essential signatures

In [2]:
essential_signatures = {
    "SIAE": "PF03629",
    "kpsD": "PF02563",
    "kpsE": "PTHR32309",
    "kpsM": "PF01061",
    "kpsT": "cd03220",
    "lic3A": "PF06002",
    "lic3B": "PF06002",
    "lst": "PF07922",
    "cpsK": "PF07922",
    "nagA": "cd00854",
    "nagB": "cd01399",
    "nanA": "PTHR42849",
    "nanC": "PF06178",
    "nanE": "PF04131",
    "nanH": "cd15482",
    "nanK": "PTHR18964",
    "nanM": "PF24996",
    "nanQ": "PF04074",
    "nanR": "cd07377",
    "nanT": "cd17316",
    "nanU": "cd08977",
    "neuA": "cd02513",
    "neuB": "cd11615",
    "neuC": "TIGR03568",
    "neuD": "cd03360",
    "neuE": "PF20471",
    "neuO": "cd04647",
    "neuS": "PF07388",
    "ompC": "PF00267",
    "ompF": "PF00267",
    "satA": "PTHR30290",
    "satB": "PTHR43163",
    "satC": "PTHR43297",
    "satD": "PTHR43776",
    "siaM": "PF06808",
    "siaT": "PF06808",
    "siaP": "PF03480",
    "siaQ": "PF04290"  
}

In [3]:
# Function to parse InterProScan output

def read_interproscan_tsv(file):
    columns = [
        "seq_id", "md5", "length", "analysis", "signature_accession",
        "signature_desc", "start", "end", "evalue", "status",
        "date", "ipr", "ipr_desc", "go", "pathway"
    ]
    try:
        df = pd.read_csv(file, sep="\t", header=None, names=columns, usecols=[0, 4])
        return df
    except Exception as e:
        print(f"Error reading {file}: {e}")
        return pd.DataFrame(columns=["seq_id", "signature_accession"])


In [4]:
# Identifying IDs that contain the essential signature

interpro_folder = "./interpro_outputs"
output_folder = "./IDs_filter1"

os.makedirs(output_folder, exist_ok=True)

for gene, signature in essential_signatures.items():
    print(f"\n=== Processing {gene} (signature: {signature}) ===")

    interpro_file = f"{gene}_interproscan.tsv"
    interpro_path = os.path.join(interpro_folder, interpro_file)

    if not os.path.exists(interpro_path):
        print(f"[ERROR] File not found: {interpro_path}. Skipping...")
        continue

    df = read_interproscan_tsv(interpro_path)

    if df.empty:
        print(f"[WARNING] Empty file for {gene}.")
        continue

    # --- FILTER BY SIGNATURE ---
    df_filtered = df[df["signature_accession"] == signature]

    # --- REMOVE DUPLICATES ---
    # Deduplicate only by seq_id
    df_filtered = df_filtered.drop_duplicates(subset=["seq_id"])

    # Output file name
    output_file = os.path.join(
        output_folder,
        f"{gene}_IDs_filter1.tsv"
    )

    df_filtered[["seq_id"]].to_csv(output_file, sep="\t", index=False, header=True)

    print(f"Rows found (after deduplication): {len(df_filtered)}")
    print(f"File saved to: {output_file}")


=== Processing SIAE (signature: PF03629) ===
Rows found (after deduplication): 113516
File saved to: ./IDs_filter1/SIAE_IDs_filter1.tsv

=== Processing kpsD (signature: PF02563) ===
Rows found (after deduplication): 12824
File saved to: ./IDs_filter1/kpsD_IDs_filter1.tsv

=== Processing kpsE (signature: PTHR32309) ===
Rows found (after deduplication): 10297
File saved to: ./IDs_filter1/kpsE_IDs_filter1.tsv

=== Processing kpsM (signature: PF01061) ===
Rows found (after deduplication): 1045
File saved to: ./IDs_filter1/kpsM_IDs_filter1.tsv

=== Processing kpsT (signature: cd03220) ===
Rows found (after deduplication): 12389
File saved to: ./IDs_filter1/kpsT_IDs_filter1.tsv

=== Processing lic3A (signature: PF06002) ===
Rows found (after deduplication): 1394
File saved to: ./IDs_filter1/lic3A_IDs_filter1.tsv

=== Processing lic3B (signature: PF06002) ===
Rows found (after deduplication): 3
File saved to: ./IDs_filter1/lic3B_IDs_filter1.tsv

=== Processing lst (signature: PF07922) ===
Ro

In [5]:
# Retrieving only InterProScan output entries corresponding to sequences
# that contain the essential signature
# Expanded code for all proteins

# === DIRECTORIES ===
signatures_folder = "./IDs_filter1"
interpro_folder = "./interpro_outputs"        # where *_interproscan.tsv are stored
output_folder = "./interpro_outputs_filter1"

os.makedirs(output_folder, exist_ok=True)

# === LOOP FOR ALL SIGNATURE FILES ===
for sig_file in glob.glob(os.path.join(signatures_folder, "*_IDs_filter1.tsv")):
    
    # extract gene name from filename
    gene = os.path.basename(sig_file).replace("_IDs_filter1.tsv", "")
    
    # corresponding InterProScan file
    interpro_file = os.path.join(interpro_folder, f"{gene}_interproscan.tsv")
    
    # if missing, skip
    if not os.path.exists(interpro_file):
        print(f"[WARNING] InterProScan file not found for {gene}")
        continue
    
    print(f"Processing {gene}...")
    
    # load files
    df_ids = pd.read_csv(sig_file, sep="\t")                    # has header
    df_inter = pd.read_csv(interpro_file, sep="\t", header=None)  # no header
    
    # first column of ID file
    ids_column = df_ids.columns[0]
    
    # filtering
    df_filtered = df_inter[df_inter[0].isin(df_ids[ids_column])]
    
    # save
    out = os.path.join(output_folder, f"{gene}_interproscan_filter1.tsv")
    df_filtered.to_csv(out, sep="\t", index=False, header=False)
    
print("\n### PROCESSING COMPLETED ###")


Processing nanR...
Processing nagB...
Processing nagA...
Processing neuO...
Processing satA...
Processing nanQ...
Processing kpsD...
Processing lst...
Processing lic3A...
Processing satC...
Processing nanA...
Processing nanT...
Processing neuS...
Processing lic3B...
Processing siaM...
Processing nanE...
Processing kpsT...
Processing satB...
Processing nanK...
Processing neuA...
Processing SIAE...
Processing neuD...
Processing neuE...
Processing neuB...
Processing nanU...
Processing cpsK...
Processing nanM...
Processing nanC...
Processing satD...
Processing kpsE...
Processing ompC...
Processing siaP...
Processing kpsM...
Processing ompF...
Processing neuC...
Processing siaT...
Processing siaQ...
Processing nanH...

### PROCESSING COMPLETED ###


## Filter 2 - Selecting IDs that have the essential signature and no extra signatures

Extra signatures are understood to be those that are not present for any of the references


In [8]:
# Generating a summary of the signatures identified in the references

path = "./interpro_refs_outputs"
files = glob.glob(os.path.join(path, "*.tsv"))

# Columns according to the InterProScan manual
columns = [
    "protein_accession",
    "md5",
    "seq_length",
    "analysis",
    "signature_accession",
    "signature_description",
    "start",
    "stop",
    "score",
    "status",
    "date",
    "ipr_accession",
    "ipr_description",
    "go_terms",
    "pathways"
]

dataframes = []

for file in files:
    # Read file
    df = pd.read_csv(file, sep="\t", names=columns, comment="#", dtype=str)

    # Extract gene name from file name
    gene_name = os.path.basename(file).split("_")[0]
    df["gene"] = gene_name

    # Select only desired columns
    df = df[[
        "gene",
        "protein_accession",
        "analysis",
        "signature_accession",
        "signature_description",
        "ipr_accession",
        "ipr_description"
    ]]

    # Remove exact duplicates (optional, avoids repetition)
    df = df.drop_duplicates()

    dataframes.append(df)

# Concatenate all genes
df_final = pd.concat(dataframes, ignore_index=True)

# Save final result with quotes in all cells
df_final.to_csv(
    "interproscan_ref_summary.csv",
    index=False,
    quoting=csv.QUOTE_ALL,
    encoding="utf-8")

In [10]:
# Filtering

interpro_folder = "./interpro_outputs_filter1"
ref_file = "./interproscan_ref_summary.csv"  
output_folder = "./suspect_IDs"

os.makedirs(output_folder, exist_ok=True)

# Read reference file
df_ref = pd.read_csv(ref_file)

# Process all .tsv files
files = [f for f in os.listdir(interpro_folder) if f.endswith(".tsv")]

print(f"Found {len(files)} files to process.")

for file in files:
    path = os.path.join(interpro_folder, file)
    gene = file.split("_")[0]

    print(f"\n=== Processing gene: {gene} ===")

    df_gene = read_interproscan_tsv(path)
    df_ref_gene = df_ref[df_ref["gene"] == gene]

    if df_ref_gene.empty:
        print(f"No reference found for {gene}. Skipping...")
        continue

    signatures_ref = set(df_ref_gene["signature_accession"].dropna().astype(str).unique())

    df_gene["signature_accession"] = df_gene["signature_accession"].astype(str)

    suspect_ids = df_gene[
        ~df_gene["signature_accession"].isin(signatures_ref)
    ]["seq_id"].unique()

    df_output = df_gene[df_gene["seq_id"].isin(suspect_ids)]

    outpath = os.path.join(output_folder, f"{gene}_suspect_IDs.tsv")
    df_output[["seq_id"]].drop_duplicates().to_csv(
        outpath,
        sep="\t",
        index=False
    )

    print(f"File generated: {outpath} ({len(df_output)} rows)")

print("\nProcessing completed! 👍")

Found 38 files to process.

=== Processing gene: nanT ===
File generated: ./suspect_IDs/nanT_suspect_IDs.tsv (37518 rows)

=== Processing gene: siaP ===


File generated: ./suspect_IDs/siaP_suspect_IDs.tsv (1649317 rows)

=== Processing gene: nanK ===
File generated: ./suspect_IDs/nanK_suspect_IDs.tsv (132474 rows)

=== Processing gene: nagB ===
File generated: ./suspect_IDs/nagB_suspect_IDs.tsv (529826 rows)

=== Processing gene: satA ===
File generated: ./suspect_IDs/satA_suspect_IDs.tsv (39436 rows)

=== Processing gene: siaM ===
File generated: ./suspect_IDs/siaM_suspect_IDs.tsv (102035 rows)

=== Processing gene: nagA ===
File generated: ./suspect_IDs/nagA_suspect_IDs.tsv (97972 rows)

=== Processing gene: satC ===
File generated: ./suspect_IDs/satC_suspect_IDs.tsv (9614 rows)

=== Processing gene: satD ===
File generated: ./suspect_IDs/satD_suspect_IDs.tsv (34756 rows)

=== Processing gene: kpsT ===
File generated: ./suspect_IDs/kpsT_suspect_IDs.tsv (9550 rows)

=== Processing gene: lst ===
File generated: ./suspect_IDs/lst_suspect_IDs.tsv (114 rows)

=== Processing gene: nanQ ===
File generated: ./suspect_IDs/nanQ_suspect_IDs.tsv 

## Compiling FASTA files


In [12]:
# Code to identify which IDs are present in IDs_filter1 but not in suspect_IDs
# In other words, retrieve IDs of sequences that contain the essential signature
# and remove IDs of sequences that contain additional signatures

dir_filter1 = "IDs_filter1"
dir_suspects = "suspect_IDs"
dir_output = "IDs_filter2"

os.makedirs(dir_output, exist_ok=True)

# List files in directories
files_filter1 = {f.split("_")[0]: f for f in os.listdir(dir_filter1)}
files_suspects = {f.split("_")[0]: f for f in os.listdir(dir_suspects)}

genes = sorted(set(files_filter1.keys()) & set(files_suspects.keys()))

for gene in genes:
    file_filter1 = os.path.join(dir_filter1, files_filter1[gene])
    file_suspects = os.path.join(dir_suspects, files_suspects[gene])

    # Read files (automatic separator detection)
    df_filter1 = pd.read_csv(file_filter1, engine="python")
    df_suspects = pd.read_csv(file_suspects, engine="python")

    # Extract IDs
    ids_filter1 = set(df_filter1["seq_id"])
    ids_suspects = set(df_suspects["seq_id"])

    # Difference
    unique_ids = sorted(ids_filter1 - ids_suspects)

    # Save output file
    out_file = os.path.join(dir_output, f"{gene}_filter2_IDs.tsv")
    df_out = pd.DataFrame({"seq_id": unique_ids})
    df_out.to_csv(out_file, sep="\t", index=False, header=True)

    print(f"{gene}: {len(unique_ids)} filter 2 IDs saved.")

print("\n✔️ Process completed!")

SIAE: 60165 filter 2 IDs saved.
cpsK: 31 filter 2 IDs saved.
kpsD: 2938 filter 2 IDs saved.
kpsE: 6598 filter 2 IDs saved.
kpsM: 1012 filter 2 IDs saved.
kpsT: 11445 filter 2 IDs saved.
lic3A: 1278 filter 2 IDs saved.
lic3B: 3 filter 2 IDs saved.
lst: 2022 filter 2 IDs saved.
nagA: 114823 filter 2 IDs saved.
nagB: 85856 filter 2 IDs saved.
nanA: 14001 filter 2 IDs saved.
nanC: 1029 filter 2 IDs saved.
nanE: 36663 filter 2 IDs saved.
nanH: 22950 filter 2 IDs saved.
nanK: 11298 filter 2 IDs saved.
nanM: 4458 filter 2 IDs saved.
nanQ: 11433 filter 2 IDs saved.
nanR: 10517 filter 2 IDs saved.
nanT: 4172 filter 2 IDs saved.
nanU: 27378 filter 2 IDs saved.
neuA: 22064 filter 2 IDs saved.
neuB: 31211 filter 2 IDs saved.
neuC: 19190 filter 2 IDs saved.
neuD: 13279 filter 2 IDs saved.
neuE: 11 filter 2 IDs saved.
neuO: 114 filter 2 IDs saved.
neuS: 22 filter 2 IDs saved.
ompC: 5201 filter 2 IDs saved.
ompF: 10057 filter 2 IDs saved.
satA: 922 filter 2 IDs saved.
satB: 4631 filter 2 IDs saved.
s

In [14]:
# Extracting FASTA sequences with essential signatures

dir_fastas = "merged_nr_fastas"                  # Original FASTA files
dir_ids = "IDs_filter1"                          # TSV files with approved seq_id
dir_output = "FASTAS_with_essential_signature"   # Final FASTA files

os.makedirs(dir_output, exist_ok=True)

# Map files by gene (before the first "_")
fastas = {f.split("_")[0]: f for f in os.listdir(dir_fastas) if f.endswith(".fasta")}
idfiles = {f.split("_")[0]: f for f in os.listdir(dir_ids) if f.endswith(".tsv")}

# Genes in common
genes = sorted(set(fastas.keys()) & set(idfiles.keys()))

print(f"Processing {len(genes)} genes...\n")

for gene in genes:

    fasta_path = os.path.join(dir_fastas, fastas[gene])
    ids_path = os.path.join(dir_ids, idfiles[gene])

    print(f"→ {gene}")

    # Read approved IDs
    df = pd.read_csv(ids_path, sep="\t")
    approved_ids = set(df["seq_id"].astype(str))

    # Read original FASTA
    records = list(SeqIO.parse(fasta_path, "fasta"))

    # Filter sequences whose header is in the list
    filtered = [
        r for r in records
        if r.id in approved_ids
    ]

    # Output file
    out_fasta = os.path.join(dir_output, f"{gene}_with_essential_signature.fasta")

    SeqIO.write(filtered, out_fasta, "fasta")

    print(f"   ✔ {len(filtered)} sequences saved to {out_fasta}")

print("\n✔ Finished!")

Processing 38 genes...

→ SIAE


   ✔ 113516 sequences saved to FASTAS_with_essential_signature/SIAE_with_essential_signature.fasta
→ cpsK
   ✔ 33 sequences saved to FASTAS_with_essential_signature/cpsK_with_essential_signature.fasta
→ kpsD
   ✔ 12824 sequences saved to FASTAS_with_essential_signature/kpsD_with_essential_signature.fasta
→ kpsE
   ✔ 10297 sequences saved to FASTAS_with_essential_signature/kpsE_with_essential_signature.fasta
→ kpsM
   ✔ 1045 sequences saved to FASTAS_with_essential_signature/kpsM_with_essential_signature.fasta
→ kpsT
   ✔ 12389 sequences saved to FASTAS_with_essential_signature/kpsT_with_essential_signature.fasta
→ lic3A
   ✔ 1394 sequences saved to FASTAS_with_essential_signature/lic3A_with_essential_signature.fasta
→ lic3B
   ✔ 3 sequences saved to FASTAS_with_essential_signature/lic3B_with_essential_signature.fasta
→ lst
   ✔ 2057 sequences saved to FASTAS_with_essential_signature/lst_with_essential_signature.fasta
→ nagA
   ✔ 123809 sequences saved to FASTAS_with_essential_signature

In [15]:
# Extracting FASTA sequences with essential signatures and without any extra signatures

dir_fastas = "merged_nr_fastas"        # Original FASTA files
dir_ids = "IDs_filter2"                  # TSV files with approved seq_id
dir_output = "FASTAS_without_extra"            # Final FASTA files

os.makedirs(dir_output, exist_ok=True)

# Map files by gene (before the first "_")
fastas = {f.split("_")[0]: f for f in os.listdir(dir_fastas) if f.endswith(".fasta")}
idfiles = {f.split("_")[0]: f for f in os.listdir(dir_ids) if f.endswith(".tsv")}

# Genes in common
genes = sorted(set(fastas.keys()) & set(idfiles.keys()))

print(f"Processing {len(genes)} genes...\n")

for gene in genes:

    fasta_path = os.path.join(dir_fastas, fastas[gene])
    ids_path = os.path.join(dir_ids, idfiles[gene])

    print(f"→ {gene}")

    # Read approved IDs
    df = pd.read_csv(ids_path, sep="\t")
    approved_ids = set(df["seq_id"].astype(str))

    # Read original FASTA
    records = list(SeqIO.parse(fasta_path, "fasta"))

    # Filter sequences whose header is in the list
    filtered = [
        r for r in records
        if r.id in approved_ids
    ]

    # Output file
    out_fasta = os.path.join(dir_output, f"{gene}_without_extra.fasta")

    SeqIO.write(filtered, out_fasta, "fasta")

    print(f"   ✔ {len(filtered)} sequences saved to {out_fasta}")

print("\n✔ Finished!")

Processing 38 genes...

→ SIAE
   ✔ 60165 sequences saved to FASTAS_without_extra/SIAE_without_extra.fasta
→ cpsK
   ✔ 31 sequences saved to FASTAS_without_extra/cpsK_without_extra.fasta
→ kpsD
   ✔ 2938 sequences saved to FASTAS_without_extra/kpsD_without_extra.fasta
→ kpsE
   ✔ 6598 sequences saved to FASTAS_without_extra/kpsE_without_extra.fasta
→ kpsM
   ✔ 1012 sequences saved to FASTAS_without_extra/kpsM_without_extra.fasta
→ kpsT
   ✔ 11445 sequences saved to FASTAS_without_extra/kpsT_without_extra.fasta
→ lic3A
   ✔ 1278 sequences saved to FASTAS_without_extra/lic3A_without_extra.fasta
→ lic3B
   ✔ 3 sequences saved to FASTAS_without_extra/lic3B_without_extra.fasta
→ lst
   ✔ 2022 sequences saved to FASTAS_without_extra/lst_without_extra.fasta
→ nagA
   ✔ 114823 sequences saved to FASTAS_without_extra/nagA_without_extra.fasta
→ nagB
   ✔ 85856 sequences saved to FASTAS_without_extra/nagB_without_extra.fasta
→ nanA
   ✔ 14001 sequences saved to FASTAS_without_extra/nanA_without_e